In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os
from IPython.display import FileLink

# 1. Read Images Data

In [2]:
%cd /home/caitsithx/courses/deeplearning1/nbs
DATA_DIR = '/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring'
TRAIN_DIR = DATA_DIR + '/train'
RESULT_DIR = DATA_DIR + '/result'

#image reseize & centering & crop 
def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

def read_images(file_paths):
    images = []
    for file_path in file_paths:
        #read image
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #resize
        if(img.shape[0] > img.shape[1]):
            tile_size = (int(img.shape[1]*256/img.shape[0]),256)
        else:
            tile_size = (256, int(img.shape[0]*256/img.shape[1]))

        #centering
        img = centering_image(cv2.resize(img, dsize=tile_size))

        #out put 224*224px 
        img = img[16:240, 16:240]
        images.append(img)
    return images

/home/caitsithx/courses/deeplearning1/nbs


## 1.1 Load Training Data Labels and file Paths 

In [3]:
df_train = pd.read_csv(DATA_DIR+'/train_labels.csv')
master = df_train
master.head()

,1,0
0,2,0
1,3,1
2,4,0
3,5,1
4,6,0


In [4]:
y = []
file_paths = []
for i in range(len(master)):
    file_paths.append( TRAIN_DIR + '/' + str(master.ix[i][0]) +'.jpg' )
    y.append(master.ix[i][1])
y = np.array(y)
print(y.shape)
print(len(file_paths))
print(file_paths[:5])

(2294,)
2294
['/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/2.jpg', '/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/3.jpg', '/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/4.jpg', '/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/5.jpg', '/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/train/6.jpg']


## 1.2 Read Training Image Data

In [ ]:
x = read_images(file_paths)
x = np.array(x)
print(x.shape)

## 1.3 Load Test Image Names and Paths

In [5]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")
img_path = DATA_DIR+"/test/unknown/"

test_names = []
file_paths = []

for i in range(len(sample_submission)):
    test_names.append(sample_submission.ix[i][0])
    file_paths.append( img_path + str(int(sample_submission.ix[i][0])) +'.jpg' )
    
test_names = np.array(test_names)
print(test_names.shape)
print(len(file_paths))
print(file_paths[:2])

(1531,)
1531
['/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/test/unknown/1.jpg', '/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/test/unknown/2.jpg']


## 1.4 Read Test Image Data

In [ ]:
test_images = read_images(file_paths)
test_images = np.array(test_images)
print(test_images.shape)

## 1.5 Save Training & Test Data

In [14]:
x = np.transpose(x, (0, 3, 1, 2))##only for channel_first_order
print(x.shape)

test_images = np.transpose(test_images, (0, 3, 1, 2)) 
print(test_images.shape)

(2294, 3, 224, 224)
(1531, 3, 224, 224)


In [ ]:
test_images = test_images.astype('float32')
test_images /= 255
x = x.astype('float32')
x /=255
print(x[:5])
print(test_images[:5])

In [39]:
np.save(open(DATA_DIR + '/train_data.npy', 'w'), x)
np.save(open(DATA_DIR + '/test_data.npy', 'w'), test_images)

## 1.6 Read Saved Training & Test Data

In [37]:
x = np.load(DATA_DIR + '/train_data.npy')
test_images = np.load(DATA_DIR + '/test_data.npy')

print(x.shape)
print(test_images.shape)

(2294, 3, 224, 224)
(1531, 3, 224, 224)


## 1.6 Split training and validation Data

In [17]:
data_num = len(y)
random_index = np.random.permutation(data_num)

x_shuffle = []
y_shuffle = []
for i in range(data_num):
    x_shuffle.append(x[random_index[i]])
    y_shuffle.append(y[random_index[i]])
    
x = np.array(x_shuffle) 
y = np.array(y_shuffle)

In [18]:
val_split_num = int(round(0.2*len(y)))
x_train = x[val_split_num:]
y_train = y[val_split_num:]
x_test = x[:val_split_num]
y_test = y[:val_split_num]

print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

('x_train', (1835, 3, 224, 224))
('y_train', (1835,))
('x_test', (459, 3, 224, 224))
('y_test', (459,))


In [19]:
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255

# 2. Build Pre-train Models

In [10]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten, Dense

img_rows, img_cols, img_channel = 224, 224, 3

Using Theano backend.
/home/caitsithx/anaconda2/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


Disabling allocation cache on cuda


In [12]:
incep_model = applications.InceptionV3(include_top=False, 
                                       weights='imagenet', 
                                       input_shape=(img_channel, img_rows, img_cols))

In [13]:
vgg16_model = applications.VGG16(weights='imagenet', 
                                 include_top=False, 
                                 input_shape=(img_channel, img_rows, img_cols))

In [20]:
vgg19_model = applications.VGG19(include_top=False, 
                                 weights='imagenet', 
                                 input_shape=(img_channel, img_rows, img_cols))

In [ ]:
res_model = applications.ResNet50(
            include_top=False, 
            weights='imagenet',
            input_tensor=None, 
            input_shape=(img_rows, img_cols, img_channel))

In [21]:
#base_model.output_shape[1:]
def get_dense_model(input_shape):
    dense_model = Sequential()
    dense_model.add(Flatten(input_shape=input_shape))
    dense_model.add(Dense(256, activation='relu'))
    dense_model.add(Dropout(0.5))
    dense_model.add(Dense(1, activation='sigmoid'))
    return dense_model

In [22]:
models = []
models.append(Model(inputs=vgg16_model.input, 
                    outputs=get_dense_model(vgg16_model.output_shape[1:])(vgg16_model.output)))
models.append(Model(inputs=vgg19_model.input, 
                    outputs=get_dense_model(vgg19_model.output_shape[1:])(vgg19_model.output)))
models.append(Model(inputs=res_model.input, 
                    outputs=get_dense_model(res_model.output_shape[1:])(res_model.output)))
#models.append(Model(inputs=incep_model.input, 
#                    outputs=get_dense_model(incep_model.output_shape[1:])(incep_model.output)))

for i, model in enumerate(models):
    model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
    print("summary of Model: " + str(i))
    model.summary()

summary of Model: 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 112, 112)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 112, 112)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 56, 56)       0

# Train Models

In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint


w_file = RESULT_DIR + '/inv-detect-pre-model{}-epoch{}.h5'

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [25]:
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch):
    if epoch <= 50:
        return 0.0001
    else: 
        return 0.00005

In [32]:
for i in range(len(models))[:1]:
    print i

0


In [35]:
batch_sizes = [40, 40, 24]
nb_epochs = 100

for i in range(len(models)):
    print ("train model: %d " % i)
    history = models[i].fit_generator(
        train_datagen.flow(x_train, y_train, batch_size=batch_sizes[i]),
        steps_per_epoch=x_train.shape[0] // batch_sizes[i],
        epochs=nb_epochs,
        callbacks=[ModelCheckpoint(w_file.format(i, nb_epochs), 
                                   monitor='val_acc', 
                                   save_best_only=True),
                                   LearningRateScheduler(lr_schedule)],
        validation_data=(x_test, y_test))

train model: 0 
Epoch 1/100
45/45 [==============================] - 72s - loss: 0.1575 - acc: 0.9411 - val_loss: 0.1098 - val_acc: 0.9586

In [40]:
preds = []
for i in range(len(models)):
    w_f = w_file.format(i, nb_epochs)
    print ("use trained model: %s " % w_f)
    models[i].load_weights(w_f)
    pred = models[i].predict(test_images)
    preds.append(pred)

print len(preds)

use trained model: 0 
use trained model: 1 
use trained model: 2 
3


In [41]:
mean_pred = np.mean(preds, axis=0)

In [43]:
print preds[0].shape
print mean_pred.shape

(1531, 1)
(1531, 1)


In [72]:
def save_result_csv(pred, prefix):
    sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")
    
    for i, test_name in enumerate(test_names):
        sample_submission.loc[sample_submission['name'] == test_name, 'invasive'] = pred[i]
    
    result_cvs_path = RESULT_DIR + "/submit2_" + str(prefix) + ".csv"
    print result_cvs_path
    sample_submission.to_csv(result_cvs_path, index=False)

In [73]:
for i in range(len(preds)):
    save_result_csv(preds[i], str(i))
    
save_result_csv(mean_pred, "mean")

/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/result/submit2_0.csv
/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/result/submit2_1.csv
/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/result/submit2_2.csv
/home/caitsithx/courses/deeplearning1/nbs/data/invasive-species-monitoring/result/submit2_mean.csv
